# Prereqs


In [ ]:
! pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 39.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.1-cp38-cp38-linux_x86_64.whl size=581428 sha256=57db0f719f1e327b101aeb484d70907280ea760f18d36129438304ca9c2516a3
  Stored in directory: /root/.cache/pip/wheels/f9/93/19/30511c4a9ae6b4937455a134c34a39e13943e2c6f46fcd2ed2
Successfully built annoy


In [ ]:
import csv
import requests
import pandas as pd

In [ ]:
import time

In [ ]:
from transformers import AutoModel, AutoTokenizer
from pprint import pprint

import faiss
import annoy
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import torch

In [ ]:
import numpy as np 

# Loading dataset and model

In [ ]:
import csv
import requests

# load training dataset
def load_data():
    CSV_URL = 'https://drive.google.com/u/0/uc?id=1Z-yb752A3o7b9dqrGt24XU0sl53FVqya&export=download'

    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        train_data = list(cr)

    print(f"Number of examples = {len(train_data)}")
    ans, noans = 0, 0
    for x in train_data:
        if x[4] == 'False':
            noans += 1
        else:
            ans += 1
    print(f"\tAnswerable questions = {ans}")
    print(f"\tNon-Answerable questions = {noans}\n")
    print("Examples:")
    for i in [0, 1000, 1300]:
        print(' | '.join(train_data[i][:2]), ' | ', train_data[i][2][:20] + '...', ' | ', ' | '.join(train_data[i][3:]))
    return train_data

In [ ]:
def load_theme_wise_data(train_data):
    theme_wise_data = {}
    for x in train_data[1:]:
        if x[1] not in theme_wise_data:
            theme_wise_data[x[1]] = {
                'para': [],
                'ques': [],
                'ans': []
            }
        if x[2] not in theme_wise_data[x[1]]['para']:
            theme_wise_data[x[1]]['para'].append(x[2])
        theme_wise_data[x[1]]['ques'].append(x[3])
        # ans contains a list -> [Para_Number, Answer_possible, Answer_text, Answer_start]
        theme_wise_data[x[1]]['ans'].append([theme_wise_data[x[1]]['para'].index(x[2])] + x[4:])
    print(f'\nTotal {len(theme_wise_data)} themes present.')
    return theme_wise_data

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
model = hub.load(module_url)

In [ ]:
train_data = load_data()
theme_wise_data = load_theme_wise_data(train_data)

Number of examples = 75056
	Answerable questions = 50126
	Non-Answerable questions = 24930

Examples:
 | Theme  |  Paragraph...  |  Question | Answer_possible | Answer_text | Answer_start
1430 | Frédéric_Chopin  |  Some modern commenta...  |  Who said Chopin's works were modeled after Bach, Beethoven, Schubert and Field? | True | ['Richard Taruskin'] | [543]
2196 | The_Legend_of_Zelda:_Twilight_Princess  |  Twilight Princess ta...  |  Who releases Bulbins from the Realm of Twilight? | False | [] | []

Total 361 themes present.


#Quora Dataset

Reference : [Quora faq datasets](https://github.com/hellohaptik/faq-datasets)

In [ ]:
tdash_ques = ['How do i get traffic for website?', 'How can I increase traffic to my site and what are some suggestions on how to get more of it?']

In [ ]:
new_tdash = {0: ['How do i get traffic for website?', '1'],
 1: ['How can I increase traffic to my site and what are some suggestions on how to get more of it?', '1']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
def create_index_annoy(embeddings, vector_length = 512, metric = 'angular', num_trees = 100):
  annoy_index = annoy.AnnoyIndex(vector_length, metric=metric)
  for i in range(len(embeddings)):
    annoy_index.add_item(i, embeddings[i])
  annoy_index.build(n_trees = num_trees)
  return annoy_index

In [ ]:
index = create_index_annoy(encoded_data)

In [ ]:
def top_k_ques(query):
    query_vector = model(query)
    results = find_similar_annoy(index,query_vector[0])

    print("\n")
    for result in results:
        print('\t',result)

In [ ]:
def find_similar_annoy(index ,embedding, num_matches=2):
  '''Finds similar items to a given embedding in the ANN index'''
  
  ids = index.get_nns_by_vector(
  embedding, num_matches, search_k=-1, include_distances=True)
  score = ids[1]
  questions = [new_tdash[id][0] for id in ids[0]]
  return [(a[0], a[1]) for a in zip(questions, score)]

In [ ]:
query = ['How can I increase the traffic on my website?']

In [ ]:
top_k_ques(query)



	 ('How can I increase traffic to my site and what are some suggestions on how to get more of it?', 0.2392665594816208)
	 ('How do i get traffic for website?', 0.3868260383605957)


In [ ]:
tdash_ques = ['How can I improve my pronunciation of English words?', 'How I can speak English with fluency?', 'What is the best way to talk English fluently?', '	How could I improve my English pronunciation?']

In [ ]:
new_tdash = {0: ['How can I improve my pronunciation of English words?', '1'],
 1: ['How I can speak English with fluency?', '1'], 2: ['What is the best way to talk English fluently?'], 3: ['How could I improve my English pronunciation?']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
index =create_index_annoy(encoded_data)

In [ ]:
query = ['What should I have to do to make my english and communication skills perfect?']

In [ ]:
top_k_ques(query)



	 ('What is the best way to talk English fluently?', 0.7073822617530823)
	 ('How I can speak English with fluency?', 0.7201260924339294)


In [ ]:
tdash_ques = ['What are some of the real life Alien Encounters (with proof)?', 'What is the best evidence for the existence of aliens?']

In [ ]:
new_tdash = {0: ['What are some of the real life Alien Encounters (with proof)?', '1'],
 1: ['What is the best evidence for the existence of aliens?', '1']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
index =create_index_annoy(encoded_data)

In [ ]:
query = ['Is there any physical proof of aliens on earth?']

In [ ]:
top_k_ques(query)



	 ('What is the best evidence for the existence of aliens?', 0.5598667860031128)
	 ('What are some of the real life Alien Encounters (with proof)?', 0.6074343919754028)


In [ ]:
tdash_ques = ['How do I stop letting my emotions control me?', 'Can we control our feelings and emotions?']

In [ ]:
new_tdash = {0: ['How do I stop letting my emotions control me?', '1'],
 1: ['Can we control our feelings and emotions?', '1']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
index =create_index_annoy(encoded_data)

In [ ]:
query = ['What is the best way to achieve emotional self control?']

In [ ]:
top_k_ques(query)



	 ('How do I stop letting my emotions control me?', 0.7092652320861816)
	 ('Can we control our feelings and emotions?', 0.8975867629051208)


In [ ]:
tdash_ques = ['Why its taking long time to answer my question in Quora? I have been asking since yesterday. I wonder why no one answer?', 'Why don\'t people on Quora answer to my questions?']

In [ ]:
new_tdash = {0: ['Why its taking long time to answer my question in Quora? I have been asking since yesterday. I wonder why no one answer?', '1'],
 1: ['Why don\'t people on Quora answer to my questions?', '1']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
index =create_index_annoy(encoded_data)

In [ ]:
query = ['Why are some questions not answered on Quora?']

In [ ]:
top_k_ques(query)



	 ("Why don't people on Quora answer to my questions?", 0.5344185829162598)
	 ('Why its taking long time to answer my question in Quora? I have been asking since yesterday. I wonder why no one answer?', 0.6736072897911072)


In [ ]:
tdash_ques = ['What are the best places to visit in Kerala? What is the best way of transportation there?', 'What are the places to visit for a honeymoon in Kerala?']

In [ ]:
new_tdash = {0: ['What are the best places to visit in Kerala? What is the best way of transportation there?', '1'],
 1: ['What are the places to visit for a honeymoon in Kerala?', '1']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
index =create_index_annoy(encoded_data)

In [ ]:
query = ['What are some good beaches to visit in Kerala?']

In [ ]:
top_k_ques(query)



	 ('What are the places to visit for a honeymoon in Kerala?', 0.6292951107025146)
	 ('What are the best places to visit in Kerala? What is the best way of transportation there?', 0.7279655337333679)


In [ ]:
tdash_ques = ['Why Indians are failing to place gold in Olympics or other international events?', 'Why India fails to get medals in Olympics?']

In [ ]:
new_tdash = {0: ['Why Indians are failing to place gold in Olympics or other international events?', '1'],
 1: ['Why India fails to get medals in Olympics?', '1']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
index =create_index_annoy(encoded_data)

In [ ]:
query = ['With a population of over 1 billion, why does India fail to shine in the Olympics?']

In [ ]:
top_k_ques(query)



	 ('Why India fails to get medals in Olympics?', 0.5529614090919495)
	 ('Why Indians are failing to place gold in Olympics or other international events?', 0.6709415316581726)


In [ ]:
tdash_ques = ['Is it possible to login to Facebook without password?', 'Is there any way to reset my Facebook personal upload email address?']

In [ ]:
new_tdash = {0: ['Is it possible to login to Facebook without password?', '1'],
 1: ['Is there any way to reset my Facebook personal upload email address?', '1']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
index =create_index_annoy(encoded_data)

In [ ]:
query = ['I forgot my Facebook email and password how can I log in?']

In [ ]:
top_k_ques(query)



	 ('Is it possible to login to Facebook without password?', 0.8110687732696533)
	 ('Is there any way to reset my Facebook personal upload email address?', 0.8526048064231873)


In [ ]:
tdash_ques = ['How do I trace the exact location of a mobile phone in India? What things are required for that', 'How can I trace phone calls from a cell phone?']

In [ ]:
new_tdash = {0: ['How do I trace the exact location of a mobile phone in India? What things are required for that', '1'],
 1: ['How can I trace phone calls from a cell phone?', '1']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
index =create_index_annoy(encoded_data)

In [ ]:
query = ['How do I track a person basing on his mobile phone?']

In [ ]:
top_k_ques(query)



	 ('How can I trace phone calls from a cell phone?', 0.7417287230491638)
	 ('How do I trace the exact location of a mobile phone in India? What things are required for that', 0.7602880001068115)


In [ ]:
tdash_ques = ['Is the potential energy of vacuum energy, virtual particles and/or dark energy infinite?', 'If dark and vacuum energy are created with expansion is there potential energy that transforms into these types of energy with expansion?']

In [ ]:
new_tdash = {0: ['Is the potential energy of vacuum energy, virtual particles and/or dark energy infinite?', '1'],
 1: ['If dark and vacuum energy are created with expansion is there potential energy that transforms into these types of energy with expansion?', '1']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
index =create_index_annoy(encoded_data)

In [ ]:
query = ['If universe expands and vacuum energy is created with it (with no limit),is there infinite potential energy/infinite vacuum energy that can be created?']

In [ ]:
top_k_ques(query)



	 ('If dark and vacuum energy are created with expansion is there potential energy that transforms into these types of energy with expansion?', 0.7097910642623901)
	 ('Is the potential energy of vacuum energy, virtual particles and/or dark energy infinite?', 0.7651801705360413)


In [ ]:
tdash_ques = ['What are the safety precautions on handling shotguns proposed by the NRA in Utah?', 'What are the safety precautions on handling shotguns proposed by the NRA in Kentucky?']

In [ ]:
new_tdash = {0: ['What are the safety precautions on handling shotguns proposed by the NRA in Utah?', '1'],
 1: ['What are the safety precautions on handling shotguns proposed by the NRA in Kentucky?', '1']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
index =create_index_annoy(encoded_data)

In [ ]:
query = ['What are the safety precautions on handling shotguns proposed by the NRA in New Hampshire?']

In [ ]:
top_k_ques(query)



	 ('What are the safety precautions on handling shotguns proposed by the NRA in Utah?', 0.39940765500068665)
	 ('What are the safety precautions on handling shotguns proposed by the NRA in Kentucky?', 0.4198141396045685)


In [ ]:
tdash_ques = ['How do I find whether my girl friend is cheating on me?', 'How do I know my partner is cheating on me?']

In [ ]:
new_tdash = {0: ['How do I find whether my girl friend is cheating on me?', '1'],
 1: ['How do I know my partner is cheating on me?', '1']} 

In [ ]:
encoded_data=model(tdash_ques)
encoded_data = np.array(encoded_data)

In [ ]:
index =create_index_annoy(encoded_data)

In [ ]:
query = ['How do I know my partner is in another relationship?']

In [ ]:
top_k_ques(query)



	 ('How do I know my partner is cheating on me?', 0.6522311568260193)
	 ('How do I find whether my girl friend is cheating on me?', 0.7995787858963013)
